# Zombie wasps, a pandemic, webcrawlers and julia

## TL;DR

How to build a webcrawler in julia.

## Motivation

The web is full of interesting bits and pieces of information. Maybe it's the current weather, stock prices, or the wikipedia article about the [wasp that goes all alien parasite facehugger on other insects](https://en.wikipedia.org/wiki/Emerald_cockroach_wasp), which you vaguely remember from one of those late night documentaries (yeah... already sorry you are reading this?). 

## Concrete example

For me, currently, this is the Covid-19 pandemic. And more specifically, how is it developing close to me. In my case, that means the city of Hamburg in Germany.

Chances are, these specific numbers are not relevant to you. But that is a good thing, you can use what you learned here and mine the website of your home city maybe. Nothing helps your brain to learn new things than generalizing the new skills and using them to solve related problems!

There is the official website of the city, that has a page for the covid-19 numbers, [hamburg.de](https://www.hamburg.de/corona-zahlen/).

The page with the numbers is in german, but don't worry, that's what our webcrawler will hopefully help us with - we can get to the numbers without having to understand all surrounding text. I will try to help out and translate what is relevant, but that will only be a minor detail when we try to find the right text to extract from.

In [21]:
using HTTP

In [22]:
url = "https://www.hamburg.de/corona-zahlen"

"https://www.hamburg.de/corona-zahlen"

css selectors

Actually, they might be called query selectors, but everyone just calls them css selectors... Oh look, a pack of wild frontenders! Hmm, what are they doing, are they encircling us? They do look kinda angry, don't they? I guess they just tried to vertically align some div the whole day or something. Ok, I guess we should leave now.

In [23]:
using Gumbo, Cascadia
using Cascadia: matchFirst

In [24]:
response = HTTP.get(url)
html = parsehtml(String(response))

HTML Document:
<!DOCTYPE >
HTMLElement{:HTML}:<HTML lang="de">
  <head></head>
  <body class="no-ads">
    HTTP/1.1 200 OK
    ServerHost: apache/portal5
    X-Frame-Options: SAMEORIGIN
    Access-Control-Allow-Origin: *
    Content-Type: text/html;charset=UTF-8
    Content-Language: de-DE
    Date: Sat, 01 Aug 2020 15:36:35 GMT
    Transfer-Encoding: chunked
    Connection: keep-alive, Transfer-Encoding
    Set-Cookie: JSESSIONID=8993885172B1E5ABA55BDDCEA3BC571D.liveWorker2; Path=/; HttpOnly
    Set-Cookie: content=13907680; Path=/servlet/segment/de/corona-zahlen/
    Set-Cookie: BIGipServerv5-webstatic-80-12-cm7=201658796.20480.0000; path=/; Httponly; Secure
    <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
    <meta charset="utf-8"/>
    <meta content="text/html" http-equiv="content-type"/>
    <script type="text/javascript">window.JS_LANG='de';    </script>
    <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
...


In [11]:
eachmatch(sel".c_chart.one .chart_legend li", html.root)

3-element Array{HTMLNode,1}:
 HTMLElement{:li}:<li>
  <span style="display:inline-block;width:.7em;height:.7em;margin-right:5px;background-color:#003063"></span>
  Bestätigte Fälle 5401
</li>


 HTMLElement{:li}:<li>
  <span style="display:inline-block;width:.7em;height:.7em;margin-right:5px;background-color:#009933"></span>
  Davon geheilt 5000
</li>


 HTMLElement{:li}:<li>
  <span style="display:inline-block;width:.7em;height:.7em;margin-right:5px;background-color:#005ca9"></span>
  Neuinfektionen 16
</li>



In [18]:
match(r"\d+", "Neuinfektionen 16")

RegexMatch("16")

In [19]:
match(r"\d+", "Neuinfektionen 16").match

"16"

In [20]:
parse(Int, match(r"\d+", "Neuinfektionen 16").match)

16

In [12]:
function parsenumbers(el)
    text = el[2].text
    parse(Int, match(r"\d+", text).match)
end

parsenumbers (generic function with 1 method)

In [13]:
map(parsenumbers, eachmatch(sel".c_chart.one .chart_legend li", html.root))

3-element Array{Int64,1}:
 5401
 5000
   16

### The end! Right?

## One more crawler